In [26]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from matplotlib.image import imread
import cv2

In [27]:
def image_retrive(path):
    folder = {f:i for i,f in enumerate(os.listdir(f'{path}'))}
    labels=[]
    M=[]
    #90 x 134
    for p_class in folder.keys():
      
        images=[f for f in os.listdir(f'{path}/{p_class}')]
        for n in range(0,len(images)):
            image=cv2.imread(f'{path}/{p_class}/{images[n]}')
            
            if image.shape[0] == 90 and image.shape[1]==134:
               
                M.append(image)
                labels.append(p_class)
            else:
                z_matrix=np.zeros((90,134,3))
                if image.shape[0]>z_matrix.shape[0] or image.shape[1] > z_matrix.shape[1]:
                    offset=abs(image.shape[0]-z_matrix.shape[0])
                    offset2=abs(image.shape[1]-z_matrix.shape[1])
                    z_matrix[:,:]=image[offset:,offset2:,:]
                M.append(z_matrix)
                labels.append(p_class)             
    return M, labels

M, labels=image_retrive('Train/Train')
T,labels2 = image_retrive('Test/Test')
V, labels3 = image_retrive('Validation/Validation')

In [28]:
M=np.array(M)
T=np.array(T)
V=np.array(V)
_, y_train=np.unique(labels, return_inverse=True)
_, y_test=np.unique(labels2, return_inverse=True)
_, y_validation=np.unique(labels3, return_inverse=True)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(90, 134,3)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Dropout(0.3),
    

    tf.keras.layers.MaxPooling2D((2, 2)),


    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


model.fit(M/255, y_train, epochs=20, validation_data=(V/255, y_validation))

Epoch 1/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 27s 618ms/step - accuracy: 0.5037 - loss: 0.9729 - val_accuracy: 0.3167 - val_loss: 2.0875
Epoch 2/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 26s 612ms/step - accuracy: 0.8496 - loss: 0.3879 - val_accuracy: 0.3000 - val_loss: 2.7165
Epoch 3/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 26s 614ms/step - accuracy: 0.9098 - loss: 0.2762 - val_accuracy: 0.3500 - val_loss: 2.5245
Epoch 4/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 27s 634ms/step - accuracy: 0.9162 - loss: 0.2443 - val_accuracy: 0.3333 - val_loss: 4.6911
Epoch 5/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 25s 597ms/step - accuracy: 0.9119 - loss: 0.2469 - val_accuracy: 0.3333 - val_loss: 4.0006
Epoch 6/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 25s 584ms/step - accuracy: 0.9499 - loss: 0.1336 - val_accuracy: 0.3000 - val_loss: 3.2452
Epoch 7/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 25s 605ms/step - accuracy: 0.9616 - loss: 0.1126 - val_accuracy: 0.3167 - val_loss: 2.5942
Epoch 8/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 26s 621ms/step - accuracy: 0.9505 - loss: 0.1547 - val_accu

In [29]:
model.evaluate(T/255, y_test, verbose=2)
model.summary()

5/5 - 1s - 114ms/step - accuracy: 0.9067 - loss: 0.4594


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_45 (Conv2D)              │ (None, 88, 132, 32)    │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 86, 130, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 84, 128, 64)    │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 84, 128, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 42, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 40, 62, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_49 (Conv2D)              │ (None, 38, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 38, 60, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 19, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 36480)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │       109,443 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 718,859 (2.74 MB)

 Trainable params: 239,619 (936.01 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 479,240 (1.83 MB)